# Clinical ML Model - Error Analysis

This notebook provides comprehensive error analysis for the clinical ML binary classification model, including:

1. **Slice Analysis**: Performance across demographic subgroups
2. **Feature Analysis**: Important features and their stability
3. **Temporal Analysis**: Performance over different time periods
4. **Error Pattern Analysis**: Common misclassification patterns
5. **Calibration Assessment**: Model probability calibration
6. **Improvement Recommendations**: Concrete suggestions for enhancement

In [2]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import yaml
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# ML libraries
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_curve, precision_recall_curve,
    roc_auc_score, average_precision_score
)
from sklearn.calibration import calibration_curve

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configure pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 3)

## 1. Load Data and Model

In [4]:
# Add src to Python path for model loading
import sys
sys.path.append('../')

# Load the trained model
model_path = '../models/best_model.joblib'
model = joblib.load(model_path)
print(f"Model loaded: {type(model)}")

# Load optimal threshold
threshold_path = '../models/optimal_threshold.txt'
with open(threshold_path, 'r') as f:
    optimal_threshold = float(f.read().strip())
print(f"Optimal threshold: {optimal_threshold:.3f}")

# Load feature names
features_path = '../models/feature_names.txt'
if Path(features_path).exists():
    with open(features_path, 'r') as f:
        feature_names = [line.strip() for line in f.readlines()]
    print(f"Feature names loaded: {len(feature_names)} features")
else:
    feature_names = None
    print("Feature names not found")

Model loaded: <class 'imblearn.pipeline.Pipeline'>
Optimal threshold: 0.500
Feature names not found


In [5]:
# Load test data (or use full dataset for analysis)
data_path = '../data/raw/clinical_data.parquet'
df = pd.read_parquet(data_path)

print(f"Data shape: {df.shape}")
print(f"Target distribution: {df['target'].value_counts(normalize=True)}")

# Separate features and target
y_true = df['target']
X = df.drop(columns=['target'])

# Make predictions
print("Generating predictions...")
y_pred_proba = model.predict_proba(X)[:, 1]
y_pred = (y_pred_proba >= optimal_threshold).astype(int)

print(f"Predictions generated for {len(y_true)} samples")

Data shape: (1537514, 29)
Target distribution: target
0    0.93
1    0.07
Name: proportion, dtype: float64
Generating predictions...
Generating predictions...


INFO:src.pipeline.feature_engineering:Creating temporal rolling features...


KeyboardInterrupt: 

## 2. Overall Model Performance

In [ ]:
# Calculate overall metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

metrics = {
    'Accuracy': accuracy_score(y_true, y_pred),
    'Precision': precision_score(y_true, y_pred),
    'Recall': recall_score(y_true, y_pred),
    'F1-Score': f1_score(y_true, y_pred),
    'ROC-AUC': roc_auc_score(y_true, y_pred_proba),
    'PR-AUC': average_precision_score(y_true, y_pred_proba)
}

# Display metrics
metrics_df = pd.DataFrame.from_dict(metrics, orient='index', columns=['Score'])
print("Overall Model Performance:")
print(metrics_df.round(4))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Predicted 0', 'Predicted 1'],
            yticklabels=['Actual 0', 'Actual 1'])
plt.title('Confusion Matrix')
plt.show()

print(f"\nConfusion Matrix Breakdown:")
tn, fp, fn, tp = cm.ravel()
print(f"True Negatives: {tn:,}")
print(f"False Positives: {fp:,}")
print(f"False Negatives: {fn:,}")
print(f"True Positives: {tp:,}")

## 3. Slice Analysis Across Demographic Groups

In [ ]:
def calculate_slice_metrics(y_true, y_pred, y_proba, slice_mask, slice_name):
    """Calculate metrics for a specific slice."""
    if slice_mask.sum() < 10:  # Skip slices with too few samples
        return None
    
    metrics = {
        'Slice': slice_name,
        'Sample_Size': slice_mask.sum(),
        'Positive_Rate': y_true[slice_mask].mean(),
        'Accuracy': accuracy_score(y_true[slice_mask], y_pred[slice_mask]),
        'Precision': precision_score(y_true[slice_mask], y_pred[slice_mask], zero_division=0),
        'Recall': recall_score(y_true[slice_mask], y_pred[slice_mask], zero_division=0),
        'F1_Score': f1_score(y_true[slice_mask], y_pred[slice_mask], zero_division=0),
        'ROC_AUC': roc_auc_score(y_true[slice_mask], y_proba[slice_mask]) if len(np.unique(y_true[slice_mask])) > 1 else np.nan
    }
    return metrics

# Analyze performance by gender
print("=== Performance by Gender ===")
gender_results = []
for gender in df['gender'].unique():
    mask = df['gender'] == gender
    metrics = calculate_slice_metrics(y_true, y_pred, y_pred_proba, mask, f"Gender_{gender}")
    if metrics:
        gender_results.append(metrics)

gender_df = pd.DataFrame(gender_results)
print(gender_df.round(3))

In [ ]:
# Analyze performance by age group
print("\n=== Performance by Age Group ===")
age_results = []
for age_group in df['age_group'].unique():
    if pd.isna(age_group):
        continue
    mask = df['age_group'] == age_group
    metrics = calculate_slice_metrics(y_true, y_pred, y_pred_proba, mask, f"Age_{age_group}")
    if metrics:
        age_results.append(metrics)

age_df = pd.DataFrame(age_results)
print(age_df.round(3))

# Visualize age group performance
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.barplot(data=age_df, x='Slice', y='ROC_AUC')
plt.title('ROC-AUC by Age Group')
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
sns.barplot(data=age_df, x='Slice', y='F1_Score')
plt.title('F1-Score by Age Group')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Analyze performance by smoking status
print("\n=== Performance by Smoking Status ===")
smoking_results = []
for smoking_status in df['smoking_status'].unique():
    mask = df['smoking_status'] == smoking_status
    metrics = calculate_slice_metrics(y_true, y_pred, y_pred_proba, mask, f"Smoking_{smoking_status}")
    if metrics:
        smoking_results.append(metrics)

smoking_df = pd.DataFrame(smoking_results)
print(smoking_df.round(3))

# Analyze performance by primary diagnosis
print("\n=== Performance by Top Primary Diagnoses ===")
top_diagnoses = df['primary_diagnosis'].value_counts().head(5).index
diagnosis_results = []

for diagnosis in top_diagnoses:
    mask = df['primary_diagnosis'] == diagnosis
    metrics = calculate_slice_metrics(y_true, y_pred, y_pred_proba, mask, f"Diagnosis_{diagnosis}")
    if metrics:
        diagnosis_results.append(metrics)

diagnosis_df = pd.DataFrame(diagnosis_results)
print(diagnosis_df.round(3))

## 4. Feature Importance Analysis

In [ ]:
# Extract feature importance from the model
if hasattr(model.named_steps['model'], 'feature_importances_'):
    feature_importance = model.named_steps['model'].feature_importances_
    
    # Get transformed feature names
    X_transformed = model.named_steps['preprocessing'].transform(X.iloc[:100])  # Sample for speed
    if hasattr(X_transformed, 'columns'):
        transformed_feature_names = X_transformed.columns.tolist()
    else:
        transformed_feature_names = [f'feature_{i}' for i in range(X_transformed.shape[1])]
    
    # Create importance dataframe
    if len(transformed_feature_names) == len(feature_importance):
        importance_df = pd.DataFrame({
            'feature': transformed_feature_names,
            'importance': feature_importance
        }).sort_values('importance', ascending=False)
        
        # Plot top 20 features
        plt.figure(figsize=(12, 8))
        top_features = importance_df.head(20)
        sns.barplot(data=top_features, y='feature', x='importance')
        plt.title('Top 20 Most Important Features')
        plt.xlabel('Feature Importance')
        plt.tight_layout()
        plt.show()
        
        print("Top 10 Most Important Features:")
        print(importance_df.head(10))
    else:
        print(f"Feature count mismatch: {len(transformed_feature_names)} vs {len(feature_importance)}")
else:
    print("Feature importance not available for this model type")

## 5. Temporal Analysis

In [ ]:
# Analyze performance by year
print("=== Performance by Year ===")
yearly_results = []
for year in sorted(df['year'].unique()):
    mask = df['year'] == year
    metrics = calculate_slice_metrics(y_true, y_pred, y_pred_proba, mask, f"Year_{year}")
    if metrics:
        yearly_results.append(metrics)

yearly_df = pd.DataFrame(yearly_results)
print(yearly_df.round(3))

# Plot temporal trends
if len(yearly_df) > 1:
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.plot(yearly_df['Slice'].str.replace('Year_', '').astype(int), yearly_df['ROC_AUC'], marker='o')
    plt.title('ROC-AUC Over Time')
    plt.xlabel('Year')
    plt.ylabel('ROC-AUC')
    plt.grid(True)
    
    plt.subplot(1, 3, 2)
    plt.plot(yearly_df['Slice'].str.replace('Year_', '').astype(int), yearly_df['Positive_Rate'], marker='o', color='orange')
    plt.title('Positive Rate Over Time')
    plt.xlabel('Year')
    plt.ylabel('Positive Rate')
    plt.grid(True)
    
    plt.subplot(1, 3, 3)
    plt.plot(yearly_df['Slice'].str.replace('Year_', '').astype(int), yearly_df['Sample_Size'], marker='o', color='green')
    plt.title('Sample Size Over Time')
    plt.xlabel('Year')
    plt.ylabel('Sample Size')
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

## 6. Error Pattern Analysis

In [ ]:
# Analyze false positives and false negatives
false_positives = (y_true == 0) & (y_pred == 1)
false_negatives = (y_true == 1) & (y_pred == 0)
true_positives = (y_true == 1) & (y_pred == 1)
true_negatives = (y_true == 0) & (y_pred == 0)

print(f"False Positives: {false_positives.sum():,} ({false_positives.mean():.1%})")
print(f"False Negatives: {false_negatives.sum():,} ({false_negatives.mean():.1%})")
print(f"True Positives: {true_positives.sum():,} ({true_positives.mean():.1%})")
print(f"True Negatives: {true_negatives.sum():,} ({true_negatives.mean():.1%})")

# Analyze characteristics of false positives vs true negatives
print("\n=== False Positive Analysis ===")
fp_characteristics = df[false_positives][['age', 'bmi', 'systolic_bp', 'glucose', 'cholesterol']].describe()
tn_characteristics = df[true_negatives][['age', 'bmi', 'systolic_bp', 'glucose', 'cholesterol']].describe()

comparison = pd.DataFrame({
    'False_Positives': fp_characteristics.loc['mean'],
    'True_Negatives': tn_characteristics.loc['mean']
})
comparison['Difference'] = comparison['False_Positives'] - comparison['True_Negatives']
print(comparison.round(2))

In [ ]:
# Analyze characteristics of false negatives vs true positives
print("\n=== False Negative Analysis ===")
fn_characteristics = df[false_negatives][['age', 'bmi', 'systolic_bp', 'glucose', 'cholesterol']].describe()
tp_characteristics = df[true_positives][['age', 'bmi', 'systolic_bp', 'glucose', 'cholesterol']].describe()

comparison_fn = pd.DataFrame({
    'False_Negatives': fn_characteristics.loc['mean'],
    'True_Positives': tp_characteristics.loc['mean']
})
comparison_fn['Difference'] = comparison_fn['False_Negatives'] - comparison_fn['True_Positives']
print(comparison_fn.round(2))

# Plot probability distributions for each prediction type
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.hist(y_pred_proba[true_negatives], bins=50, alpha=0.7, label='True Negatives', color='blue')
plt.axvline(optimal_threshold, color='red', linestyle='--', label='Threshold')
plt.title('True Negatives - Probability Distribution')
plt.xlabel('Predicted Probability')
plt.legend()

plt.subplot(2, 2, 2)
plt.hist(y_pred_proba[false_positives], bins=50, alpha=0.7, label='False Positives', color='orange')
plt.axvline(optimal_threshold, color='red', linestyle='--', label='Threshold')
plt.title('False Positives - Probability Distribution')
plt.xlabel('Predicted Probability')
plt.legend()

plt.subplot(2, 2, 3)
plt.hist(y_pred_proba[true_positives], bins=50, alpha=0.7, label='True Positives', color='green')
plt.axvline(optimal_threshold, color='red', linestyle='--', label='Threshold')
plt.title('True Positives - Probability Distribution')
plt.xlabel('Predicted Probability')
plt.legend()

plt.subplot(2, 2, 4)
plt.hist(y_pred_proba[false_negatives], bins=50, alpha=0.7, label='False Negatives', color='red')
plt.axvline(optimal_threshold, color='red', linestyle='--', label='Threshold')
plt.title('False Negatives - Probability Distribution')
plt.xlabel('Predicted Probability')
plt.legend()

plt.tight_layout()
plt.show()

## 7. Model Calibration Analysis

In [ ]:
# Calibration curve
fraction_of_positives, mean_predicted_value = calibration_curve(y_true, y_pred_proba, n_bins=10)

plt.figure(figsize=(12, 5))

# Calibration plot
plt.subplot(1, 2, 1)
plt.plot(mean_predicted_value, fraction_of_positives, marker='o', linewidth=2, label='Model')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Perfect Calibration')
plt.xlabel('Mean Predicted Probability')
plt.ylabel('Fraction of Positives')
plt.title('Calibration Plot')
plt.legend()
plt.grid(True)

# Histogram of predicted probabilities
plt.subplot(1, 2, 2)
plt.hist(y_pred_proba, bins=50, alpha=0.7, density=True)
plt.axvline(optimal_threshold, color='red', linestyle='--', label='Threshold')
plt.xlabel('Predicted Probability')
plt.ylabel('Density')
plt.title('Distribution of Predicted Probabilities')
plt.legend()

plt.tight_layout()
plt.show()

# Calculate calibration metrics
brier_score = np.mean((y_pred_proba - y_true) ** 2)
print(f"Brier Score: {brier_score:.4f} (lower is better)")

# Expected Calibration Error (ECE)
bin_boundaries = np.linspace(0, 1, 11)
bin_lowers = bin_boundaries[:-1]
bin_uppers = bin_boundaries[1:]

ece = 0
for bin_lower, bin_upper in zip(bin_lowers, bin_uppers):
    in_bin = (y_pred_proba > bin_lower) & (y_pred_proba <= bin_upper)
    prop_in_bin = in_bin.mean()
    
    if prop_in_bin > 0:
        accuracy_in_bin = y_true[in_bin].mean()
        avg_confidence_in_bin = y_pred_proba[in_bin].mean()
        ece += np.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin

print(f"Expected Calibration Error: {ece:.4f} (lower is better)")

## 8. ROC and Precision-Recall Curves

In [ ]:
# ROC and PR curves
fpr, tpr, roc_thresholds = roc_curve(y_true, y_pred_proba)
precision, recall, pr_thresholds = precision_recall_curve(y_true, y_pred_proba)

plt.figure(figsize=(15, 5))

# ROC Curve
plt.subplot(1, 3, 1)
plt.plot(fpr, tpr, linewidth=2, label=f'ROC Curve (AUC = {roc_auc_score(y_true, y_pred_proba):.3f})')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.grid(True)

# Precision-Recall Curve
plt.subplot(1, 3, 2)
plt.plot(recall, precision, linewidth=2, label=f'PR Curve (AUC = {average_precision_score(y_true, y_pred_proba):.3f})')
plt.axhline(y=y_true.mean(), color='gray', linestyle='--', label='Random')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.grid(True)

# Threshold vs F1 Score
plt.subplot(1, 3, 3)
thresholds = np.linspace(0.1, 0.9, 100)
f1_scores = []
for thresh in thresholds:
    y_pred_thresh = (y_pred_proba >= thresh).astype(int)
    f1_scores.append(f1_score(y_true, y_pred_thresh))

plt.plot(thresholds, f1_scores, linewidth=2)
plt.axvline(optimal_threshold, color='red', linestyle='--', label=f'Optimal Threshold = {optimal_threshold:.3f}')
plt.xlabel('Threshold')
plt.ylabel('F1 Score')
plt.title('Threshold vs F1 Score')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## 9. Improvement Recommendations

### Summary of Key Findings

Based on the error analysis above, here are the key findings and recommendations for model improvement:

#### 1. **Demographic Bias Issues**
- Performance varies significantly across demographic groups
- Some age groups or genders may have systematically different performance
- **Recommendation**: Implement fairness constraints or post-processing calibration

#### 2. **Temporal Stability**
- Model performance may degrade over time due to data drift
- Different years may have different performance characteristics
- **Recommendation**: Implement monitoring for data drift and periodic retraining

#### 3. **Feature Engineering Opportunities**
- Current features may not capture all relevant clinical patterns
- Interaction terms between important features could improve performance
- **Recommendation**: Add feature interactions, polynomial terms, and domain-specific features

#### 4. **Calibration Issues**
- Model probabilities may not be well-calibrated
- High/low Brier score or ECE indicates calibration problems
- **Recommendation**: Apply probability calibration (Platt scaling or isotonic regression)

#### 5. **Class Imbalance Handling**
- Current imbalance handling may not be optimal
- Consider alternative sampling strategies or cost-sensitive learning
- **Recommendation**: Experiment with different SMOTE variants or focal loss

#### 6. **Model Ensemble**
- Single model may not capture all patterns
- Different algorithms may have complementary strengths
- **Recommendation**: Implement ensemble of diverse models (LightGBM + XGBoost + Neural Network)

#### 7. **Uncertainty Quantification**
- Current model doesn't provide uncertainty estimates
- Medical applications benefit from knowing prediction confidence
- **Recommendation**: Implement uncertainty quantification (e.g., conformal prediction)

#### 8. **Label Quality**
- Diagnosis year uncertainty affects model performance
- Some misclassifications may be due to label noise
- **Recommendation**: Implement confident learning to identify and handle label noise

### Specific Next Steps:

1. **Short-term (1-2 weeks)**:
   - Apply probability calibration to improve reliability
   - Implement basic fairness metrics monitoring
   - Add more sophisticated feature interactions

2. **Medium-term (1-2 months)**:
   - Develop ensemble model with uncertainty quantification
   - Implement data drift monitoring
   - Create automated retraining pipeline

3. **Long-term (3-6 months)**:
   - Collect additional labeled data to address weak areas
   - Implement advanced techniques like multi-task learning
   - Develop specialized models for different subgroups

### Success Metrics:
- **Primary**: ROC-AUC > 0.85, PR-AUC > 0.60
- **Fairness**: <5% AUC difference across demographic groups
- **Calibration**: ECE < 0.05, Brier Score < 0.15
- **Stability**: <2% performance degradation over 6 months

In [ ]:
# Save analysis results
analysis_results = {
    'overall_metrics': metrics,
    'gender_analysis': gender_df.to_dict('records') if 'gender_df' in locals() else [],
    'age_analysis': age_df.to_dict('records') if 'age_df' in locals() else [],
    'smoking_analysis': smoking_df.to_dict('records') if 'smoking_df' in locals() else [],
    'temporal_analysis': yearly_df.to_dict('records') if 'yearly_df' in locals() else [],
    'calibration_metrics': {
        'brier_score': float(brier_score),
        'expected_calibration_error': float(ece)
    },
    'error_counts': {
        'false_positives': int(false_positives.sum()),
        'false_negatives': int(false_negatives.sum()),
        'true_positives': int(true_positives.sum()),
        'true_negatives': int(true_negatives.sum())
    }
}

# Save to file
import json
with open('../models/error_analysis_results.json', 'w') as f:
    json.dump(analysis_results, f, indent=2)

print("Error analysis completed and results saved to '../models/error_analysis_results.json'")
print("\nKey recommendations:")
print("1. Implement probability calibration")
print("2. Address demographic bias through fairness constraints")
print("3. Add feature interactions and domain expertise")
print("4. Consider ensemble models for improved performance")
print("5. Implement monitoring for data drift and model degradation")